In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### LangGraph 주요 컴포넌트

```
스테이트: 그래프의 상태 표현
```

In [3]:
import operator
from typing import Annotated
from langchain_core.pydantic_v1 import BaseModel, Field

class State(BaseModel):
  query: str = Field(..., description='사용자의 질문')
  current_role: str = Field(default='', description='선정된 답변 역할')
  messages: Annotated[list[str], operator.add] = Field(defualt=[], description='답변 이력')
  current_judge: bool = Field(default=False, description='품질 검사 결과')
  judgement_reason: str = Field(deafult='', description='품질 검사 판정 이유')

c:\Users\user\anaconda3\envs\skn\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

```
노드: 그래프를 구성하는 처리 단위
```
노드의 지정 방법

In [5]:
# 함수 또는 Runnable만 지정하는 예
# 이 경우 노드 이름 'answering_node'가 됨
# 방법1 : workflow.add_node(answering_node)

# answering_node이라는 이름의 노드를 정의
# 노드 내 처리는 answering_ndoe 함수가 수행
# 방법2 : workflow.add_node('answering_node', answering_node)

노드의 구현 방법

In [6]:
from typing import Any

def answering_node(state: State) -> dict[str, Any]:
  query = state.query
  role = state.current_role

  # 사용자 질문 내용과 선택된 역할을 바탕으로 답변을 생성하는 로직
  generated_message = "...생성 처리..."

  # 생성된 답변으로 스테이트 업데이트
  return {'messages': [generated_message]}

def check_node(state: State) -> dict[str, Any]:
  query = state.query
  message = state.messages[-1]

  # 사용자의 질문 내용과 답변 내용에서 품질 검사를 수행하는 처리
  judge = "...판정 결과..."
  reason = "...판정 이유..."

  # 생성된 답변으로 스테이트 업데이트
  return {"current_judge": judge, 'judgement_reason': reason}

```
엣지: 노드 간의 연결
```

엔트리 포인트

In [7]:
# 그래프 시작 노드 지정 엣지

workflow.set_entry_point('selection')
workflow.add_edge('START', 'selection')

엣지

In [8]:
# 특정 노드에서 다른 노드로 무조건 전이하는 엣지
workflow.add_edge('selection', 'answering')

조건부 엣지

In [9]:
# 조건에 기반하여 전이할 노드를 결정하는 엣지
from langgraph.graph import END

workflow.add_conditional_edges(
  'check',
  lambda state: state.current_judge,
  {True: END, False: 'selection'}
)

```
컴파일된 그래프
```

In [10]:
# 정의된 그래프는 complie함수를 통해 실행 가능한 CompliedGraph 인스턴스로 변환
complied = workflow.compile()

ValueError: Found edge starting at unknown node 'START'

invoke 함수

In [ ]:
# invoke 함수를 사용하면 그래프 내의 모든 처리가 실행된 후 최종 값이 반환
initial_state = State(query='생성형 AI에 관해 알려주세요')
result = complied.invoke(initial_state)

ainvoke 함수

In [ ]:
# 작동은 invoke 함수와 동일하지만, ainvoke 함수를 사용하면 비동기 함수로 실행할 수 있음
initial_state = State(query='생성 AI에 관해 알려주세요')
result = await complied.ainvoke(initial_state)

stream 함수

In [ ]:
# 노드 실행 시 스테이트를 순차적으로 가져올 수 있음
initial_state = State(query='생성 AI에 관해 알려주세요')
for step in compiled.stream(initial_state):
  print(step)

### 실습: Q&A 애플리케이션

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)

c:\Users\user\anaconda3\envs\skn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


역할 정의

In [ ]:
ROLES = {
    "1": {
        "name": "일반 지식 전문가",
        "description": "폭넓은 분야의 일반적인 질문에 답변",
        "details": "폭넓은 분야의 일반적인 질문에 대해 정확하고 이해하기 쉬운 답변을 제공하세요."
    },
    "2": {
        "name": "생성형 AI 제품 전문가",
        "description": "생성형 AI와 관련 제품, 기술에 관한 전문적인 질문에 답변",
        "details": "생성형 AI와 관련 제품, 기술에 관한 전문적인 질문에 대해 최신 정보와 깊은 통찰력을 제공하세요."
    },
    "3": {
        "name": "카운슬러",
        "description": "개인적인 고민이나 심리적인 문제에 대해 지원 제공",
        "details": "개인적인 고민이나 심리적인 문제에 대해 공감적이고 지원적인 답변을 제공하고, 가능하다면 적절한 조언도 해주세요."
    }
}

스테이트 정의

In [ ]:
import operator
from typing import Annotated
from langchain_core.pydantic_v1 import BaseModel, Field

class State(BaseModel):
  query: str = Field(..., description='사용자의 질문')
  current_role: str = Field(default='', description='선정된 답변 역할')
  messages: Annotated[list[str], operator.add] = Field(default=[], description='답변이력')
  current_judge: bool = Field(default=False, description='품질 체크 결과')
  judgement_reason: str = Field(default='', description='품질 체크 판정 이유')

selection 노드 구현

In [ ]:
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
  query = state.query
  role_options = '\n'.join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])

  prompt = ChatPromptTemplate.from_template("""질문을 분석하고 가장 적합한 답변 담당 역할을 선택하세요.

선택지:
{role_options}

답변은 선택지의 번호(1, 2, 또는 3)만 반환하세요.

질문: {query}""".strip())
  
  # 선택지의 번호만 반환하기를 기대하므로 max_tokens 값을 1로 변경
  chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
  role_number = chain.invoke({"role_options": role_options, "query": query})

  selected_role = ROLES[role_number.strip()]["name"]
  return {"current_role": selected_role}

answering 노드 구현

In [ ]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""당신은 {role}로서 답변하세요. 다음 질문에 대해 당신의 역할에 기반한 적절한 답변을 제공하세요.

역할 상세:
{role_details}

질문: {query}

답변:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [ ]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="판정 결과")
    reason: str = Field(default="", description="판정 이유")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""다음 답변의 품질을 체크하고, 문제가 있으면 'False', 문제가 없으면 'True'로 답변하세요.
또한, 그 판정 이유도 설명하세요.

사용자의 질문: {query}
답변: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [ ]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [ ]:
# selection 노드에서 처리 시작
workflow.set_entry_point("selection")

In [ ]:
# selection 노드에서 answering 노드로
workflow.add_edge("selection", "answering")
# answering 노드에서 check 노드로
workflow.add_edge("answering", "check")

In [ ]:
from langgraph.graph import END

# check 노드에서 다음 노드로의 전환에 조건부 엣지 정의
# state.current_judge 값이 True면 END 노드로, False면 selection 노드로
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [ ]:
compiled = workflow.compile()

In [ ]:
initial_state = State(query="생성형 AI에 관해 알려주세요")
result = compiled.invoke(initial_state)

c:\Users\user\anaconda3\envs\skn\Lib\site-packages\langchain_openai\chat_models\base.py:1915: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [ ]:
result

{'query': '생성형 AI에 관해 알려주세요',
 'current_role': '생성형 AI 제품 전문가',
 'messages': ['생성형 AI(Generative AI)는 데이터를 학습하여 새로운 콘텐츠를 생성할 수 있는 인공지능 기술을 말합니다. 이 기술은 텍스트, 이미지, 음악, 비디오 등 다양한 형태의 콘텐츠를 만들어낼 수 있으며, 최근 몇 년간 급격히 발전해 왔습니다. \n\n**1. 작동 원리:**  \n생성형 AI는 주로 딥러닝 기술을 사용하여 대량의 데이터를 학습합니다. 그 중에서도 GAN(Generative Adversarial Networks)과 VAE(Variational Autoencoders) 같은 모델들이 널리 사용됩니다. GAN은 두 개의 신경망(생성자와 판별자)이 경쟁하면서 성능을 향상시키는 구조로, VAE는 데이터의 분포를 모델링하여 새로운 샘플을 생성합니다.\n\n**2. 활용 분야:**  \n- **텍스트 생성:** GPT-3, GPT-4와 같은 모델은 자연어 처리에 활용되어 인간과 유사한 텍스트를 생성합니다. 이를 통해 글쓰기, 번역, 요약 등의 작업을 자동화할 수 있습니다.\n- **이미지 생성:** DALL-E와 같은 모델은 텍스트 설명에 기반하여 이미지를 생성합니다. 이는 광고, 디자인, 예술 등 다양한 분야에서 활용될 수 있습니다.\n- **음악 및 비디오 생성:** 생성형 AI는 음악 작곡이나 비디오 편집에도 사용되며, 창작의 새로운 가능성을 열어줍니다.\n\n**3. 장점과 단점:**  \n장점으로는 창의적인 작업을 보조하고 생산성을 높여주는 것이 있으며, 사용자 맞춤형 콘텐츠 생성이 가능하다는 점이 있습니다. 그러나 단점으로는 생성된 콘텐츠의 품질이 항상 일정하지 않거나, 윤리적 문제(예: 저작권, 편향된 데이터 문제)가 발생할 수 있다는 점이 있습니다.\n\n**4. 미래 전망:**  \n생성형 AI는 다양한 산업에서 혁신을 일으킬 것으로 기대되며, 인공지능의 발전과 함께 더욱 정교하고 인간다운 창작 

In [ ]:
print(result['messages'][-1])

생성형 AI(Generative AI)는 데이터를 학습하여 새로운 콘텐츠를 생성할 수 있는 인공지능 기술을 말합니다. 이 기술은 텍스트, 이미지, 음악, 비디오 등 다양한 형태의 콘텐츠를 만들어낼 수 있으며, 최근 몇 년간 급격히 발전해 왔습니다. 

**1. 작동 원리:**  
생성형 AI는 주로 딥러닝 기술을 사용하여 대량의 데이터를 학습합니다. 그 중에서도 GAN(Generative Adversarial Networks)과 VAE(Variational Autoencoders) 같은 모델들이 널리 사용됩니다. GAN은 두 개의 신경망(생성자와 판별자)이 경쟁하면서 성능을 향상시키는 구조로, VAE는 데이터의 분포를 모델링하여 새로운 샘플을 생성합니다.

**2. 활용 분야:**  
- **텍스트 생성:** GPT-3, GPT-4와 같은 모델은 자연어 처리에 활용되어 인간과 유사한 텍스트를 생성합니다. 이를 통해 글쓰기, 번역, 요약 등의 작업을 자동화할 수 있습니다.
- **이미지 생성:** DALL-E와 같은 모델은 텍스트 설명에 기반하여 이미지를 생성합니다. 이는 광고, 디자인, 예술 등 다양한 분야에서 활용될 수 있습니다.
- **음악 및 비디오 생성:** 생성형 AI는 음악 작곡이나 비디오 편집에도 사용되며, 창작의 새로운 가능성을 열어줍니다.

**3. 장점과 단점:**  
장점으로는 창의적인 작업을 보조하고 생산성을 높여주는 것이 있으며, 사용자 맞춤형 콘텐츠 생성이 가능하다는 점이 있습니다. 그러나 단점으로는 생성된 콘텐츠의 품질이 항상 일정하지 않거나, 윤리적 문제(예: 저작권, 편향된 데이터 문제)가 발생할 수 있다는 점이 있습니다.

**4. 미래 전망:**  
생성형 AI는 다양한 산업에서 혁신을 일으킬 것으로 기대되며, 인공지능의 발전과 함께 더욱 정교하고 인간다운 창작 능력을 갖추게 될 것입니다. 이는 교육, 의료, 엔터테인먼트 등 여러 분야에서 새로운 기회를 창출할 것입니다.

이와 같은 생성형 AI의 기술과 응용은 계속해서 발전하

In [31]:
# Mermaid 형식으로 그래프 시각화 (pygraphviz 불필요)
from IPython.display import display, Markdown

mermaid_graph = compiled.get_graph().draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_graph}\n```"))

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	selection(selection)
	answering(answering)
	check(check)
	__end__([<p>__end__</p>]):::last
	__start__ --> selection;
	answering --> check;
	check -. &nbsp;True&nbsp; .-> __end__;
	check -. &nbsp;False&nbsp; .-> selection;
	selection --> answering;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [33]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 그래프의 상태 정의
class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# 메시지를 추가하는 노드 함수
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="당신은 최소한의 응답을 하는 대화 에이전트입니다.")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

# LLM에서의 응답을 추가하는 노드 함수
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [34]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("체크포인트 데이터:")
    pprint(checkpoint_tuple.checkpoint)
    print("\n메타데이터:")
    pprint(checkpoint_tuple.metadata)

In [43]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 그래프 설정
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# 체크포인터 설정
checkpointer = MemorySaver()

# 그래프 컴파일
compiled_graph = graph.compile(checkpointer=checkpointer)

In [44]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다! 찹쌀떡을 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 45, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVBOWij4mBU7EEwug8fJHAhajJQDx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4d8a787c-d100-49dc-87ab-a03005342395-0', usage_metadata={'input_tokens': 45, 'output_tokens': 16, 'total_tokens': 61, 'input_token_details': {'audio': 0,

In [37]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b3011-9b19-6402-8002-a203a485c077'}}, checkpoint={'v': 4, 'ts': '2025-10-27T06:49:35.589273+00:00', 'id': '1f0b3011-9b19-6402-8002-a203a485c077', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.2098028561977191', 'query': '00000000000000000000000000000002.0.2098028561977191', 'messages': '00000000000000000000000000000004.0.5468092109718535', 'branch:to:add_message': '00000000000000000000000000000003.0.029370591939699486', 'branch:to:llm_response': '00000000000000000000000000000004.0.5468092109718535'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.3945438607681522'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000002.0.2098028561977191'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000003.0.029370591939699486'}}, 'updated_channels': ['messages'], 'channel_v

In [38]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는군요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 45, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVBKpBmVulVxhoflS9I233kvFzyYe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c8c445df-57f5-445f-bd30-f4d1a463b9a7-0', usage_metadata={'input_tokens': 45, 'output_tokens': 16, 'tot

In [39]:
user_query = State(query="제가 좋아하는 것이 뭔지 기억나세요?")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '제가 좋아하는 것이 뭔지 기억나세요?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는군요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 45, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVBKpBmVulVxhoflS9I233kvFzyYe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c8c445df-57f5-445f-bd30-f4d1a463b9a7-0', usage_metadata={'input_tokens': 45, 'output_tokens': 16, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache

In [40]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b3012-c236-6281-8006-83955a6bf60b'}}, checkpoint={'v': 4, 'ts': '2025-10-27T06:50:06.534105+00:00', 'id': '1f0b3012-c236-6281-8006-83955a6bf60b', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.5728597736301598', 'query': '00000000000000000000000000000006.0.5728597736301598', 'messages': '00000000000000000000000000000008.0.765763461519953', 'branch:to:add_message': '00000000000000000000000000000007.0.45225145459812544', 'branch:to:llm_response': '00000000000000000000000000000008.0.765763461519953'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.7585605724937053'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000006.0.5728597736301598'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000007.0.45225145459812544'}}, 'updated_channels': ['messages'], 'channel_value

In [41]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 좋아하는 것은 찹쌀떡입니다. 기억해 주세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다. 찹쌀떡을 좋아하시는군요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 45, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVBKpBmVulVxhoflS9I233kvFzyYe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c8c445df-57f5-445f-bd30-f4d1a463b9a7-0', usage_metadata={'input_tokens': 45, 'output_tokens': 16, 'tot

In [42]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="제가 좋아하는 것은 뭔가요?")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '제가 좋아하는 것은 뭔가요?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 좋아하는 것은 뭔가요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='모르겠어요. 당신의 취향에 대해 더 알려주시면 좋겠어요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 38, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CVBLryJYsdFsZP7She2x1H94l25h2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--13a70f63-a024-4780-b41d-f194d1d66e0a-0', usage_metadata={'input_tokens': 38, 'output_tokens': 19, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read'